In [ ]:
import pandas as pd 
import numpy as np
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
from datetime import datetime

In [ ]:
!ls "/content/drive/My Drive/DS4A - Team 23"

'all wsb comments.csv'	        r_wallstreetbets_daily_comments_12.csv
'clean wsb comments.ipynb'      r_wallstreetbets_daily_comments.csv
 Datasets.gdoc		        r_wallstreetbets_daily.csv
 final_one_big.csv	        r_wallstreetbets_daily_full_cleaned.csv
 getComments.py		        r_wallstreetbets_daily_full.csv
 getRedditDaily.py	        r_wallstreetbets_daily_urls.csv
 popularityData.ipynb	        ScrapeReddit-R.R
 popularity_export	        Team23_ProjectOutline.gdoc
 prawTest.ipynb		        Team23_ProjectProposal2.gdoc
 Robinhood_API_article-master   Team23_ProjectProposal.gdoc
 r_wallstreetbets_1000.csv      vixcurrent.csv


In [ ]:
df = pd.read_csv('/content/drive/My Drive/DS4A - Team 23/r_wallstreetbets_daily_comments_cleaned.csv')

In [ ]:
df.tail()

,topic,body,timestamp
224158,7nm292,You should stick this; so everyone can see. Th...,2018-01-02 16:30:14
224159,7nev10,Does anyone wanna suck my dick,2018-01-01 18:53:14
224160,7nev10,Buying a fuckton of options on weed stocks sin...,2018-01-01 19:05:53
224161,7nev10,Happy MU year !,2018-01-02 00:34:33
224162,7nev10,[deleted],2018-01-01 16:08:18


In [ ]:
#tesla df 
keywords = ["TSLA", "tesla"]
tesla_dic = {"body":[], "timestamp":[]}

for index, row in df.iterrows():
  for word in keywords:
    if word in row["body"]:
      tesla_dic["body"].append(row["body"])
      tesla_dic["timestamp"].append(row["timestamp"])
      break 

In [ ]:
tesla_df = pd.DataFrame.from_dict(tesla_dic)

In [ ]:
tesla_df.head()

,body,timestamp
0,I miss when TSLA nearly doubled in a 20 day pe...,2020-09-29 15:02:34
1,I just realised I might have a gambling proble...,2020-09-29 23:41:29
2,I'm retarded for selling call spreads on TSLA ...,2020-09-29 20:04:50
3,My TSLA weeklies are fucked. Account is about ...,2020-09-29 16:20:07
4,TSLA $450 end of day,2020-09-29 22:11:51


In [ ]:
for index, row in tesla_df.iterrows():
  try:
    time_str = row["timestamp"][:10]
    tesla_df.at[index, "timestamp"] = datetime.strptime(time_str, "%Y-%m-%d")
  except ValueError:
    continue

In [ ]:
tesla_df.head()

,body,timestamp
0,I miss when TSLA nearly doubled in a 20 day pe...,2020-09-29 00:00:00
1,I just realised I might have a gambling proble...,2020-09-29 00:00:00
2,I'm retarded for selling call spreads on TSLA ...,2020-09-29 00:00:00
3,My TSLA weeklies are fucked. Account is about ...,2020-09-29 00:00:00
4,TSLA $450 end of day,2020-09-29 00:00:00


In [ ]:
import nltk
nltk.download("vader_lexicon")
nltk.download('punkt')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# initialize VADER 
vad = SentimentIntensityAnalyzer()

In [ ]:
test_mesg = tesla_df["body"][2]
print (test_mesg)

I'm retarded for selling call spreads on TSLA and PTON in consecutive weeks. Just mouth-drooling retardation.


In [ ]:
scores = vad.polarity_scores(test_mesg)
scores

{'compound': -0.5719, 'neg': 0.198, 'neu': 0.802, 'pos': 0.0}

In [ ]:
# iterate through the whole dataframe and calculate compound vad score
vad_compound = []
for index, row in tesla_df.iterrows():
  msg = row["body"]
  scores = vad.polarity_scores(msg)
  compound = scores["compound"]
  vad_compound.append(compound)
tesla_df["vader score"] = vad_compound

In [ ]:
tesla_df.head()

,body,timestamp,vader score
0,I miss when TSLA nearly doubled in a 20 day pe...,2020-09-29 00:00:00,-0.1531
1,I just realised I might have a gambling proble...,2020-09-29 00:00:00,-0.4767
2,I'm retarded for selling call spreads on TSLA ...,2020-09-29 00:00:00,-0.5719
3,My TSLA weeklies are fucked. Account is about ...,2020-09-29 00:00:00,-0.6597
4,TSLA $450 end of day,2020-09-29 00:00:00,0.0000


In [ ]:
tesla_df.to_csv("/content/drive/My Drive/DS4A - Team 23/tesla_sentiment.csv")